In [58]:
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import SQLContext, Row

from dateutil.parser import parse
from datetime import datetime

# conf = SparkConf().setAppName("wind-sfpd")
# sc = SparkContext(conf=conf)
sqlc = SQLContext(sc)

# df = sqlc.read.format('com.databricks.spark.csv').\
# options(header='true', inferschema='true').\
# load('/home/oxclo/datafiles/wind2014/*.csv')

df = spark.read.format('csv').options(header='true').load('/home/oxclo/datafiles/wind2014/*.csv')

def date_and_hour(s):
    dt = parse(s.replace('?',' '))
    hour = dt.hour
    return dt.strftime("%Y-%m-%d")+":" +str(hour)

tidied = df.rdd.map(lambda r: Row(station = r.Station_ID, datehour =date_and_hour(r.Interval_End_Time), \
temp=r.Ambient_Temperature_Deg_C, wind=r.Wind_Velocity_Mtr_Sec)).toDF()

nonulls = tidied.filter(tidied.temp.isNotNull()).filter(tidied.wind.isNotNull())

numbered = nonulls.rdd.map(lambda row: Row(station=row.station, datehour=row.datehour, wind=float(row.wind), temp=float(row.temp))).toDF()
numbered.show(10)

# tidied.registerTempTable("tempwind")

# averages = sqlc.sql("select station, datehour, avg(temp) as avgtemp, avg(wind) as avgwind from tempwind group by station, datehour")

averages = numbered.groupBy(['station','datehour']).agg({'temp':'avg', 'wind':'avg'})


averages.show(10)

cleanedaverages = averages.rdd.map(lambda row: Row(station=row.station, datehour=row.datehour, temp=row['avg(temp)'], wind=row['avg(wind)'])).toDF()
cleanedaverages.show(10)
 


+-------+------------+-----+-----+
|station|    datehour| wind| temp|
+-------+------------+-----+-----+
|   SF15|2014-01-04:0|0.671|3.466|
|   SF15|2014-01-04:0|1.353|2.941|
|   SF15|2014-01-04:0|0.907|2.606|
|   SF15|2014-01-04:0|0.641|3.226|
|   SF15|2014-01-04:0|  0.3|3.637|
|   SF15|2014-01-04:0|0.193|3.383|
|   SF15|2014-01-04:0|  0.0|2.998|
|   SF15|2014-01-04:0|  0.0|2.765|
|   SF15|2014-01-04:0|  0.0| 2.28|
|   SF15|2014-01-04:0|  0.0|2.649|
+-------+------------+-----+-----+
only showing top 10 rows

+-------+-------------+------------------+------------------+
|station|     datehour|         avg(temp)|         avg(wind)|
+-------+-------------+------------------+------------------+
|   SF15| 2014-01-01:7| 4.964458333333332|0.8799791666666664|
|   SF15|2014-05-03:15|27.309583333333336|2.4722083333333336|
|   SF15| 2014-05-04:8|21.153750000000002|2.4801249999999997|
|   SF15|2014-06-01:14| 33.20849999999999|2.1222833333333333|
|   SF15|2014-09-05:10| 25.05479166666667|2.163604

In [53]:
idf = spark.read.format('csv').options(header='true').load('/home/oxclo/datafiles/incidents/sfpd.csv')
withyx2014 = idf.filter(idf.X.isNotNull()).filter(idf.Y.isNotNull()).filter(idf.Date.contains('2014'))
tidy = withyx2014.rdd.map(lambda row: Row(datehour = date_and_hour(row.Date+" "+row.Time),yx=[float(row.Y),float(row.X)])).toDF()
tidy.show(10)

+-------------+--------------------+
|     datehour|                  yx|
+-------------+--------------------+
|2014-12-31:23|[37.7799444052046...|
|2014-12-31:23|[37.7959918873778...|
|2014-12-31:23|[37.7887772719153...|
|2014-12-31:23|[37.7900712300699...|
|2014-12-31:23|[37.8073339305952...|
|2014-12-31:23|[37.7898859908661...|
|2014-12-31:23|[37.7898859908661...|
|2014-12-31:23|[37.7899968072158...|
|2014-12-31:23|[37.7626702770872...|
|2014-12-31:23|[37.7869408998805...|
+-------------+--------------------+
only showing top 10 rows



In [55]:
from numpy import array
from scipy import spatial


def locate(l,index,locations):
    distance,i = index.query(l)
    return locations[i]

def map_yx_to_station(yx):
    return locate(yx, \
        spatial.KDTree(array( \
        [[37.7816834,-122.3887657],\
        [37.7469112,-122.4821759],\
        [37.7411022,-120.804151],\
        [37.4834543,-122.3187302],\
        [37.7576436,-122.3916382],\
        [37.7970013,-122.4140409],\
        [37.748496,-122.4567461],\
        [37.7288155,-122.4210133],\
        [37.5839487,-121.9499339],\
        [37.7157156,-122.4145311],\
        [37.7329613,-122.5051491],\
        [37.7575891,-122.3923824],\
        [37.7521169,-122.4497687]])),
        ["SF18", "SF04", "SF15", "SF17", "SF36", "SF37",\
        "SF07", "SF11", "SF12", "SF14", "SF16", "SF19", "SF34"] )

# sqlc.udf.register('map_yx_to_station',map_yx_to_station)
                  
                  
tidy.registerTempTable('incidents')

# withstations = sqlc.sql("select map_yx_to_station(yx) as station, datehour from incidents")
withstations = tidy.rdd.map(lambda row: Row(station=map_yx_to_station(row.yx), datehour=row.datehour)).toDF()
# print(map_yx_to_station([37.7816834,-122.3887657]))
withstations.show(10)

withstations.registerTempTable('stationincidents')

incidentcount = sqlc.sql("select station, datehour, count(1) as incidents from stationincidents group by station, datehour")
incidentcount.show(100)

+-------+-------------+
|station|     datehour|
+-------+-------------+
|   SF37|2014-12-31:23|
|   SF37|2014-12-31:23|
|   SF37|2014-12-31:23|
|   SF18|2014-12-31:23|
|   SF37|2014-12-31:23|
|   SF37|2014-12-31:23|
|   SF37|2014-12-31:23|
|   SF37|2014-12-31:23|
|   SF34|2014-12-31:23|
|   SF37|2014-12-31:23|
+-------+-------------+
only showing top 10 rows

+-------+-------------+---------+
|station|     datehour|incidents|
+-------+-------------+---------+
|   SF04|2014-12-31:18|        2|
|   SF14| 2014-12-29:8|        1|
|   SF19|2014-12-26:22|        4|
|   SF34| 2014-12-25:0|        1|
|   SF37| 2014-12-22:4|        1|
|   SF37|2014-12-18:15|        8|
|   SF36| 2014-12-18:5|        1|
|   SF36|2014-12-16:21|        2|
|   SF18|2014-12-16:20|        3|
|   SF14| 2014-12-16:1|        2|
|   SF04|2014-12-15:15|        3|
|   SF37|2014-12-14:18|        8|
|   SF34|2014-12-14:18|        2|
|   SF19| 2014-12-14:1|        1|
|   SF14|2014-12-12:12|        2|
|   SF34| 2014-12-12:6|   

In [59]:
joined = cleanedaverages.join(incidentcount, ['station', 'datehour'], 'outer')
joined.show(10)

+-------+-------------+------------------+------------------+---------+
|station|     datehour|              temp|              wind|incidents|
+-------+-------------+------------------+------------------+---------+
|   SF04|2014-01-10:19|              null|              null|        1|
|   SF04|2014-02-03:13|11.799791666666664|1.7297083333333336|        1|
|   SF04|2014-02-06:16|13.486944444444447| 2.406222222222222|        1|
|   SF04|2014-02-07:20|            8.4175|1.3776666666666666|        1|
|   SF04|2014-02-09:23|              null|              null|        1|
|   SF04| 2014-02-17:7|              null|              null|        1|
|   SF04| 2014-02-21:3|              null|              null|        1|
|   SF04|2014-03-23:19|              null|              null|        1|
|   SF04|2014-04-07:10|12.728541666666667|2.6746041666666667|        1|
|   SF04|2014-04-12:11|              null|              null|        2|
+-------+-------------+------------------+------------------+---

In [61]:
zeroed = joined.rdd.map(lambda row: Row(station = row.station, datehour=row.datehour, temp = row.temp, wind = row.wind, incidents = row.incidents if row.incidents  else 0)).toDF()
# zeroed.show(100)
final = zeroed.filter(zeroed.temp.isNotNull()).filter(zeroed.wind.isNotNull()).filter(zeroed.temp!=0)



In [63]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.stat import Statistics

vecs = final.rdd.map(lambda row: Vectors.dense([row.temp,row.wind,row.incidents]))

print(Statistics.corr(vecs))

[[1.         0.24453833 0.06069799]
 [0.24453833 1.         0.24087844]
 [0.06069799 0.24087844 1.        ]]
